In [ ]:
# some simple commands

In [ ]:
import torch


In [ ]:
x = torch.tensor([1., 2.])
y = torch.tensor([3., 2.])
x

In [ ]:
x * y

In [ ]:
x = torch.tensor([
                  [1., 2.], 
                  [3., 2.], 
                  [0., 4.],
                 ])
print(x[0][1])
x[0][1].item()

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
torch

In [ ]:
# y = torch.tensor([3., 2.], device="cuda")


In [ ]:
# example with mnist


In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.autograd import Variable

import torch.nn.functional as F

import torch.utils.data as data

In [2]:
train = pd.read_csv("../data/mnist/train.csv")

In [3]:
train_labels = train.label.values
train = train.drop(columns="label").values.reshape(len(train_labels), 1, 28, 28)

In [4]:
# reshape for a NN

In [6]:
# build neural network

In [7]:
train_labels

array([1, 0, 1, ..., 7, 6, 9])

In [53]:
class MNISTClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        num_labels = len(set(train_labels))
        img_height = 28
        img_width = 28 
        s_in = img_height * img_width
        s2 = 392
        s3 = 196
        s4 = 98
        s_out = num_labels
        
        self.fc1 = nn.Linear(s_in, s2) # fully connected
        self.fc2 = nn.Linear(s2, s3)
        self.fc3 = nn.Linear(s3, s4)
        self.fc4 = nn.Linear(s4, s_out)
        # regularise NN to avoid overfitting
        self.dropout = nn.Dropout(p=0.2) # 20% of nodes will be randomly 
                                         # not used during training iterations 
                                         # makes network more robust since each node
                                          # will not be used in every iteration.
    def forward(self, x):
        """forward pass"""
        x = x.view(x.shape[0], -1) # flatten img size to one long vector
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = F.log_softmax(self.fc4(x), dim=1)
        return x

            


In [54]:
model = MNISTClassifier()
loss_function = nn.NLLLoss() # negative log likelyhood -> loss = -log(y)
opt = optim.Adam(model.parameters(), lr=0.001)

In [55]:
opt

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

In [56]:
loss_function

NLLLoss()

In [57]:
model

MNISTClassifier(
  (fc1): Linear(in_features=784, out_features=392, bias=True)
  (fc2): Linear(in_features=392, out_features=196, bias=True)
  (fc3): Linear(in_features=196, out_features=98, bias=True)
  (fc4): Linear(in_features=98, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [58]:
# pytorch requires features to be floats for gradients
y = torch.Tensor(train_labels).long() # long -> 64bit integers
# enforce labels as integers
X = torch.Tensor(train.astype(float)) # 32bit floating-point numbers 



In [63]:
# Training
epochs = 50
for epoch in range(epochs):
    # transform to variable so that they have the "backward()" method
    images = Variable(X)
    labels = Variable(y)
    
    # set all gradients to zero first
    # gradients are calculated cumulatively on each back prop which is useful in RNNs
    # but in this case we want the gradient of the epoch only
    # so after each pass re-set it to zero
    opt.zero_grad() 
    # make  the forward pass:
    outputs = model(images)
    # calculate the loss
    loss = loss_function(outputs, labels)
    # backprop the loss
    loss.backward()
    # update the model paramates using the optimiser
    opt.step()
    # print the total loss
    print(f"epoch {epoch}/{epochs} Loss: {loss.data.item()}")
    
    

epoch 0/50 Loss: 0.25746870040893555
epoch 1/50 Loss: 0.2520117461681366
epoch 2/50 Loss: 0.25036194920539856
epoch 3/50 Loss: 0.2453850656747818
epoch 4/50 Loss: 0.23968103528022766
epoch 5/50 Loss: 0.23763632774353027
epoch 6/50 Loss: 0.23409509658813477
epoch 7/50 Loss: 0.22732368111610413
epoch 8/50 Loss: 0.22185702621936798
epoch 9/50 Loss: 0.22170798480510712
epoch 10/50 Loss: 0.2187337428331375
epoch 11/50 Loss: 0.21716316044330597
epoch 12/50 Loss: 0.21428647637367249
epoch 13/50 Loss: 0.21122561395168304
epoch 14/50 Loss: 0.20733249187469482
epoch 15/50 Loss: 0.1997411698102951
epoch 16/50 Loss: 0.19777585566043854
epoch 17/50 Loss: 0.19691821932792664
epoch 18/50 Loss: 0.19304496049880981
epoch 19/50 Loss: 0.19227662682533264
epoch 20/50 Loss: 0.19132159650325775
epoch 21/50 Loss: 0.18614822626113892
epoch 22/50 Loss: 0.184334397315979
epoch 23/50 Loss: 0.1840767115354538
epoch 24/50 Loss: 0.18102434277534485
epoch 25/50 Loss: 0.17897947132587433
epoch 26/50 Loss: 0.174726366